In [19]:
import qmcpy as qp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cProfile
import pstats

In [20]:
test_seed = 24
def natural_process(n, point_num, seed=None, cores = None):
    l = qp.Lattice(dimension=n, seed=seed,order="natural" ,process=True,max_workers=cores)
    points = l.gen_samples(point_num) 
    return points
def linear_process(n, point_num, seed=None, cores = None):
    l = qp.Lattice(dimension=n, seed=seed,order="linear" ,process=True,max_workers=cores)
    points = l.gen_samples(point_num) 
    return points
def mps_process(n, point_num, seed=None, cores = None):
    l = qp.Lattice(dimension=n, seed=seed,order="mps" ,process=True,max_workers=cores)
    points = l.gen_samples(point_num) 
    return points
def natural_thread(n, point_num, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="natural" ,thread=True)
    points = l.gen_samples(point_num) 
    return points
def linear_thread(n, point_num, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="linear" ,thread=True)
    points = l.gen_samples(point_num) 
    return points
def mps_thread(n, point_num, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="mps" ,thread=True)
    points = l.gen_samples(point_num) 
    return points

In [21]:
trials = 5
dimension_list = [50, 100, 200, 500, 1400, 3500]
n = 10
data = {'Dimensions':dimension_list}
dimension_frame = pd.DataFrame(data)
dimension_frame = dimension_frame.set_index('Dimensions')
names = set()

run_list = [(natural_thread, "natural thread"),(linear_thread,"linear thread"),(mps_thread, "mps thread"), (natural_process, "natural process"),(linear_process,"linear process"),(mps_process, "mps process")]

for method,method_name in run_list:

    name = method_name.split(" ")

    names.add(name[0])

    times = []
    
    for d in dimension_list:
        profiler = cProfile.Profile()
        profiler.enable()
        for _ in range(trials):
            method(d, 2 ** n,seed=test_seed)
        profiler.disable()
        
        stats = pstats.Stats(profiler)
            
        process_time = stats.total_tt

        times.append(process_time)
        
    dimension_frame[method_name] = times  
stats.sort_stats('cumulative')


names = list(names)  

The comparison list compares the different runtimes between each function. The functions you want to compare are put into a tuple like (a, b). The runtime of function a is divided by b. Giving the speed-up factor with regard to the second function. This means that an end result of above 1 would mean function b is faster than function a. Then these results are added to the original frame.

In [22]:
comparison= [('mps thread', 'mps process'), ('linear thread', 'linear process'), ('natural thread', 'natural process')]

def speed_up_cal(data_frame, compare_list, column_list, end_suffix):
    result = []
    for seq,pro in compare_list:
        result = []
        for x in column_list:
            a = data_frame[seq][x]
            b = data_frame[pro][x]
            result.append(a/b)
        data_frame[seq + " " + end_suffix] = result

speed_up_cal(dimension_frame, comparison, dimension_list, "thread/process speed-up")
#print(dimension_frame)


In [23]:
to_str_list = lambda array: [str(x) for x in array]
y_axis = dimension_frame["natural thread/process speed-up"].to_numpy()
x_axis = to_str_list(dimension_list)
plt.bar(x_axis, y_axis)
plt.xlabel('Dimensions')
plt.ylabel('Speed-Up Factor')
plt.title('Natural method multithreading and multiprocessing')
plt.show()

y_axis = dimension_frame["linear process speed-up"].to_numpy()
x_axis = to_str_list(dimension_list)
plt.bar(x_axis, y_axis)
plt.xlabel('Dimensions')
plt.ylabel('Speed-Up Factor')
plt.title('Linear method multithreading and multiprocessing')
plt.show()

y_axis = dimension_frame["mps process speed-up"].to_numpy()
x_axis = to_str_list(dimension_list)
plt.bar(x_axis, y_axis)
plt.xlabel('Dimensions')
plt.ylabel('Speed-Up Factor')
plt.title('Magic point shop multithreading and multiprocessing')
plt.show()

KeyError: 'natural thread/process speed-up'

In [ ]:
d = 10
points_list = [10, 100, 1000, 10_000, 15_000]
records = {'Points':points_list}
points_frame = pd.DataFrame(records)
points_frame = points_frame.set_index('Points')
names = set()


for method, method_name in run_list:
    name = method_name.split(" ")

    names.add(name[0])


    times = []
    
    for p in points_list:
        profiler = cProfile.Profile()
        profiler.enable()
        for _ in range(trials):
            method(2 ** d, p, seed=test_seed)
        profiler.disable()
    
        stats = pstats.Stats(profiler)
        
        process_time = stats.total_tt


        times.append(process_time)
    
    points_frame[method_name] = times  
stats.sort_stats('cumulative')


names = list(names)     

#Speed up calculation
speed_up_cal(points_frame, comparison, points_list, "point process and thread speed-up")

#Uncomment to view updated frame
#print(points_frame)

In [ ]:
y_axis = points_frame["natural point process and thread speed-up"].to_numpy()
x_axis = to_str_list(points_list)
plt.bar(x_axis, y_axis)
plt.xlabel('Points')
plt.ylabel('Speed-Up Factor')
plt.title('Natural method multithreading and multiprocessing')
plt.show()

y_axis = points_frame["linear point process and thread speed-up"].to_numpy()
x_axis = to_str_list(points_list)
plt.bar(x_axis, y_axis)
plt.xlabel('Dimensions')
plt.ylabel('Speed-Up Factor')
plt.title('Linear method multithreading and multiprocessing')
plt.show()


y_axis = points_frame["mps point process and thread speed-up"].to_numpy()
x_axis = to_str_list(points_list)
plt.bar(x_axis, y_axis)
plt.xlabel('Dimensions')
plt.ylabel('Speed-Up Factor')
plt.title('Magic point shop multithreading and multiprocessing')
plt.show()
